### !!!
### Keep in mind, that the google colab session will crash after executing the first cell. This is necessary to load open-instruct as an editable package. Just continue by executing the next cell after the session has crashed.
### !!!

In [ ]:
import time


# google colab related stuff
def is_running_on_colab():
    try:
        import google.colab
        return True
    except ImportError:
        return False

if is_running_on_colab():
    import os
    os.chdir("/content")
    !git clone https://github.com/lukas-hinterleitner/master-thesis.git

    os.chdir("/content/master-thesis")
    !git submodule init
    !git submodule update

    !pip uninstall torchvision torchaudio fastai timm -y
    !pip install -r requirements.txt
    os.kill(os.getpid(), 9)

Cloning into 'master-thesis'...
remote: Enumerating objects: 273, done.
remote: Counting objects: 100% (157/157), done.
remote: Compressing objects: 100% (110/110), done.
remote: Total 273 (delta 86), reused 105 (delta 40), pack-reused 116 (from 1)
Receiving objects: 100% (273/273), 45.82 MiB | 39.50 MiB/s, done.
Resolving deltas: 100% (130/130), done.
Submodule 'submodules/open-instruct' (https://github.com/allenai/open-instruct) registered for path 'submodules/open-instruct'
Cloning into '/content/master-thesis/submodules/open-instruct'...
Submodule path 'submodules/open-instruct': checked out 'bdc3fa6e79c455b1b9ef6c4170b688ed4aa0c5e0'
Found existing installation: torchvision 0.20.0+cu121
Uninstalling torchvision-0.20.0+cu121:
  Successfully uninstalled torchvision-0.20.0+cu121
Found existing installation: torchaudio 2.5.0+cu121
Uninstalling torchaudio-2.5.0+cu121:
  Successfully uninstalled torchaudio-2.5.0+cu121
Obtaining file:///content/master-thesis/submodules/open-instruct (from

In [1]:
# google colab related stuff
def is_running_on_colab():
    try:
        import google.colab
        return True
    except ImportError:
        return False

if is_running_on_colab():
    import os
    os.chdir("/content/master-thesis/code")

    from google.colab import output
    output.enable_custom_widget_manager()

In [2]:
from transformers import AutoModelForCausalLM, AutoTokenizer
from datasets import load_from_disk
from tqdm.notebook import tqdm

import torch
import random
import gc
import time

import numpy as np
import pandas as pd

from config import hf_model_id, lima_paraphrased_dataset_path, get_dataset_config, get_gradient_similarity_file_path

from utilities.preprocessing import prepare_dataset
from utilities.gradient_operations import get_gradients, get_flattened_weight_vector

[2024-11-10 16:59:34,137] [INFO] [real_accelerator.py:219:get_accelerator] Setting ds_accelerator to cuda (auto detect)


In [3]:
# Set seeds for reproducibility
torch.manual_seed(42)
np.random.seed(42)
random.seed(42)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

In [4]:
torch.cuda.empty_cache()
gc.collect()

30

In [5]:
model = AutoModelForCausalLM.from_pretrained(hf_model_id)
tokenizer = AutoTokenizer.from_pretrained(hf_model_id)

print(tokenizer.eos_token_id)
print(tokenizer.pad_token_id)

config.json:   0%|          | 0.00/632 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/4.71G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/5.37k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.12M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/65.0 [00:00<?, ?B/s]

50279
1


In [6]:
dataset_config = get_dataset_config(model)
dataset_config

DatasetConfig(chat_template='tulu', preference_chosen_key='chosen', preference_rejected_key='rejected', sft_messages_key='messages', binary_messages_key='messages', label='binary_labels', convert_preference_to_binary_dataset=False, max_token_length=2048, max_prompt_token_length=None, sanity_check=False, sanity_check_max_samples=100, batched=False, load_from_cache_file=True, num_proc=12, train_only_on_prompt=True, ncols=2)

In [7]:
use_gpu = True

device = torch.device('cuda' if torch.cuda.is_available() and use_gpu else 'cpu')
device

device(type='cuda')

In [8]:
model.to(device)
model.eval() # set to evaluation because we don't need to update weights

OlmoForCausalLM(
  (model): OlmoModel(
    (embed_tokens): Embedding(50304, 2048, padding_idx=1)
    (layers): ModuleList(
      (0-15): 16 x OlmoDecoderLayer(
        (self_attn): OlmoSdpaAttention(
          (q_proj): Linear(in_features=2048, out_features=2048, bias=False)
          (k_proj): Linear(in_features=2048, out_features=2048, bias=False)
          (v_proj): Linear(in_features=2048, out_features=2048, bias=False)
          (o_proj): Linear(in_features=2048, out_features=2048, bias=False)
          (rotary_emb): OlmoRotaryEmbedding()
        )
        (mlp): OlmoMLP(
          (gate_proj): Linear(in_features=2048, out_features=8192, bias=False)
          (up_proj): Linear(in_features=2048, out_features=8192, bias=False)
          (down_proj): Linear(in_features=8192, out_features=2048, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): OlmoLayerNorm()
        (post_attention_layernorm): OlmoLayerNorm()
      )
    )
    (norm): OlmoLayerNorm()
  )
  (

In [9]:
model.num_parameters()

1176764416

In [10]:
dataset = load_from_disk(lima_paraphrased_dataset_path)

In [11]:
dataset.column_names

['id', 'messages', 'paraphrased_messages']

In [12]:
# create two datasets for original messages and paraphrased messages

original_dataset_columns = ["id", "messages"]
paraphrased_dataset_columns = ["id", "paraphrased_messages"]

original_dataset = dataset.select_columns(original_dataset_columns)
paraphrased_dataset = dataset.select_columns(paraphrased_dataset_columns)

In [13]:
print(original_dataset.column_names)
print(paraphrased_dataset_columns)

['id', 'messages']
['id', 'paraphrased_messages']


In [14]:
# rename paraphrased messages to messages since open-instruct encode_sft_example only works with 'messages'key
paraphrased_dataset = paraphrased_dataset.rename_column("paraphrased_messages", "messages")

In [15]:
original_dataset

Dataset({
    features: ['id', 'messages'],
    num_rows: 988
})

In [16]:
paraphrased_dataset

Dataset({
    features: ['id', 'messages'],
    num_rows: 988
})

In [26]:
sample_size = 20 # original_dataset.num_rows
sample_size

20

In [27]:
original_dataset_tokenized = prepare_dataset(dataset=dataset, tokenizer=tokenizer, model=model, sample_size=sample_size)

Tokenizing and reformatting instruction data:   0%|          | 0/20 [00:00<?, ? examples/s]

Filter:   0%|          | 0/20 [00:00<?, ? examples/s]

In [28]:
paraphrased_dataset_tokenized = prepare_dataset(dataset=paraphrased_dataset, tokenizer=tokenizer, model=model, sample_size=sample_size)

Tokenizing and reformatting instruction data:   0%|          | 0/20 [00:00<?, ? examples/s]

Filter:   0%|          | 0/20 [00:00<?, ? examples/s]

In [29]:
start_time = time.time()

data = []

gradients = dict()

original_ids = set()
paraphrased_ids = set()

progress_wrapper = tqdm(original_dataset_tokenized, desc="Calculating gradients and corresponding similarities")

for original in progress_wrapper:
    original_id = original["id"][0][0]
    original_ids.add(original_id)

    original_gradients = get_gradients(model, original, device)
    original_flattened_gradients = get_flattened_weight_vector(original_gradients)

    for paraphrased in paraphrased_dataset_tokenized:
        paraphrased_id = paraphrased["id"][0][0]
        paraphrased_ids.add(paraphrased_id)

        progress_wrapper.set_description(desc=f"Processing original ({original_id}) and paraphrased ({paraphrased_id})")

        paraphrased_gradients = get_gradients(model, paraphrased, device)
        paraphrased_flattened_gradients = get_flattened_weight_vector(paraphrased_gradients)

        similarity = original_flattened_gradients.dot(paraphrased_flattened_gradients).item()
        data.append((original_id, paraphrased_id, similarity))

progress_wrapper.set_description("Calculating gradients and corresponding similarities")

end_time = time.time()
execution_time = end_time - start_time
print(f"Execution time: {execution_time} seconds")

Calculating gradients and corresponding similarities:   0%|          | 0/20 [00:00<?, ?it/s]

Execution time: 1065.295674085617 seconds


In [30]:
df = pd.DataFrame(data, columns=['original_id', 'paraphrased_id', 'value'])
df_pivot = df.pivot(index='original_id', columns='paraphrased_id', values='value')
df_pivot = df_pivot.reindex(index=sorted(original_ids), columns=sorted(paraphrased_ids))

In [31]:
df_pivot

paraphrased_id,lima_0,lima_1,lima_10,lima_11,lima_12,lima_13,lima_14,lima_15,lima_16,lima_17,lima_18,lima_19,lima_2,lima_3,lima_4,lima_5,lima_6,lima_7,lima_8,lima_9
original_id,,,,,,,,,,,,,,,,,,,,
lima_0,704.572021,7.378688,11.461467,12.120822,18.128651,22.099636,6.780462,44.812889,27.117548,13.710809,22.579773,21.729382,26.238413,5.326207,-2.626504,35.678165,30.139153,7.959303,6.076345,8.485947
lima_1,-3.731161,197.466629,12.230677,13.620668,10.966014,0.252617,5.198039,26.114071,15.929282,9.757661,8.133455,6.573665,-1.583301,-5.860563,-2.444476,5.129814,15.379052,-12.913933,5.850690,5.515174
lima_10,24.279398,7.066396,690.942078,21.412390,9.884799,26.556271,9.687924,33.067459,28.441954,29.322430,9.918931,17.085218,19.193981,6.010208,-1.215360,21.607300,13.291119,15.296870,44.949654,0.306385
lima_11,11.765073,8.633123,11.026361,275.722687,4.089410,-1.336506,10.571337,32.076054,29.813339,9.991892,1.225374,-0.875437,16.728966,4.716888,15.499039,20.712608,16.657940,13.541716,11.697985,9.997189
lima_12,22.049469,5.885386,6.101056,1.379657,282.684845,-4.566419,17.275784,33.573536,23.106266,1.961985,4.333669,31.534882,19.186174,-4.832311,7.066750,7.814498,11.545047,4.826401,-3.169100,0.732262
lima_13,37.223312,11.876122,17.526442,7.855901,57.621433,380.768677,-4.262122,68.318329,35.388733,11.544577,14.263382,16.682079,14.653450,22.357937,-2.479635,5.007300,35.098022,6.769525,34.623363,20.330606
lima_14,4.918539,4.930912,10.846545,9.903999,16.727095,12.838992,458.224579,28.426016,50.470036,11.161835,11.464369,17.788528,0.854584,-2.163798,0.188994,24.338095,0.405535,-5.581327,26.889410,-10.534447
lima_15,40.746933,18.665174,29.513676,24.577553,50.159138,32.227482,39.014606,547.299866,71.611191,19.945551,27.825581,25.614035,53.939739,0.416932,15.009769,31.623775,40.519810,10.406205,47.468437,2.686373
lima_16,40.143883,14.668742,23.587147,26.588387,6.933215,31.908892,35.974327,68.808434,783.278320,32.690968,9.085128,35.049385,13.432009,9.141353,11.337858,67.491585,23.455008,7.400437,34.523525,9.427513


In [34]:
os.chdir("/content/master-thesis/code")
df_pivot.to_csv(get_gradient_similarity_file_path(sample_size), index=True, header=True)

In [35]:
# google colab related stuff to publish data automatically to github repository
if is_running_on_colab():
    from google.colab import userdata
    os.chdir("/content/master-thesis")

    !git config user.email "{userdata.get('GIT_EMAIL')}"
    !git config user.name "{userdata.get('GIT_NAME')}"

    !git add .

    commit_message = "\"executing gradient similarity for sample size " + str(sample_size) + "\""
    !git commit -m '"{commit_message}"'
    !git remote set-url origin "https://{userdata.get('GIT_TOKEN')}@github.com/lukas-hinterleitner/master-thesis.git"
    !git push

[main c3d2faa] ""executing gradient similarity for sample size 20""
 1 file changed, 21 insertions(+)
 create mode 100644 data/gradient_similarity/sample_size_20.csv
Enumerating objects: 8, done.
Counting objects: 100% (8/8), done.
Delta compression using up to 12 threads
Compressing objects: 100% (5/5), done.
Writing objects: 100% (5/5), 4.28 KiB | 4.28 MiB/s, done.
Total 5 (delta 2), reused 0 (delta 0), pack-reused 0
remote: Resolving deltas: 100% (2/2), completed with 2 local objects.
To https://github.com/lukas-hinterleitner/master-thesis.git
   d40ac03..c3d2faa  main -> main


In [ ]:
# todo: investigate how olmo uses a single training iteration, check masking
# todo: add filtering with regard to open instruct (threshold for similarity)
# todo: ranking between sampling
# todo: tf-idf -> term-frequency inverse-document-frequency
# todo: think about explainability vs. similarity

In [ ]:
# random projections to reduce weight vector size
# compare ranking to other algorithms: bm25, tf-idf, (rouge optionally)